<a href="https://colab.research.google.com/github/iiVSX/SWE3032_41_team10/blob/main/New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
from google.colab import files
uploaded = files.upload()

Saving Train1.xlsx to Train1 (1).xlsx


In [60]:
import pandas as pd
dataset = pd.read_excel('Train1 (1).xlsx')

happy_data = dataset[dataset['감정'] == '행복']
sad_data = dataset[dataset['감정'] == '슬픔']
fear_data = dataset[dataset['감정'] == '공포']
neutral_data = dataset[dataset['감정'] == '중립']
aversion_data = dataset[dataset['감정'] == '혐오']
surprise_data = dataset[dataset['감정'] == '놀람']
anger_data = dataset[dataset['감정'] == '분노']

print(len(happy_data))
print(len(sad_data))
print(len(fear_data))
print(len(neutral_data))
print(len(aversion_data))
print(len(surprise_data))
print(len(anger_data))


7887
7450
7027
7753
7892
7351
7510


In [61]:
from sklearn.utils import resample

minority_data_size = len(fear_data)

happy_data_resampled = resample(happy_data, replace=False, n_samples=minority_data_size, random_state=42)
sad_data_resampled = resample(sad_data, replace=False, n_samples=minority_data_size, random_state=42)
neutral_data_resampled = resample(neutral_data, replace=False, n_samples=minority_data_size, random_state=42)
aversion_data_resampled = resample(aversion_data, replace=False, n_samples=minority_data_size, random_state=42)
surprise_data_resampled = resample(surprise_data, replace=False, n_samples=minority_data_size, random_state=42)

balanced_data = pd.concat([happy_data_resampled, sad_data_resampled, fear_data, neutral_data_resampled, aversion_data_resampled, surprise_data_resampled])

print(dataset)
print(balanced_data)

                                                 문장  감정
0              아 진짜! 사무실에서 피지 말라니깐! 간접흡연이 얼마나 안좋은데!  분노
1      그럼 직접흡연하는 난 얼마나 안좋겠니? 안그래? 보면 꼭... 지 생각만 하고.  혐오
2                                           손님 왔어요.  중립
3                                           손님? 누구?  중립
4                                    몰라요. 팀장님 친구래요.  중립
...                                             ...  ..
52865                                어떤 홈 트레이닝이 있어?  슬픔
52866                                 나 엘리베이터에 갇혔어.  공포
52867                             자취방 엘리베이턴데 정전인가봐.  공포
52868                               나 드디어 프로젝트 끝났어!  행복
52869                                    걱정해줘서 고마워.  중립

[52870 rows x 2 columns]
                                           문장  감정
25028                       집에서 밥상에 정말도움이됩니다.  행복
47680  마라톤 대회를 출전했는데 내 기존 기록보다 훨씬 좋은 기록을 세웠어!  행복
43454                  알았어. 친구들과 약속도 좀 잡아 볼게.  행복
51423       야, 나 드디어 레포트 다 끝났어! 진짜 너무 좋아, 너무.  행복
42856                                   응 없어.  행복
..

In [48]:
!pip install transformers seqeval[gpu]
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
# GPU 사용
device = torch.device("cuda")

In [ ]:
from transformers import ElectraModel, ElectraTokenizer

model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [84]:
from sklearn.preprocessing import LabelEncoder

class NSMCDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.dataset['감정'])

    def __getitem__(self, idx):
        row = self.dataset.iloc[idx, 0:2].values
        text = row[0]
        label = self.labels[idx]

        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            max_length=256,
            pad_to_max_length=True,
            add_special_tokens=True
        )

        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        return input_ids, attention_mask, torch.tensor(label)

    def __len__(self):
        return len(self.dataset)

In [85]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(balanced_data, test_size=0.2, random_state=42)

train_dataset = NSMCDataset(train_data, tokenizer)
test_dataset = NSMCDataset(test_data, tokenizer)

In [86]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [87]:
# model's layer
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [88]:
batch_size = 16
num_epochs = 10
learning_rate = 2e-5

In [89]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [90]:
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = F.cross_entropy

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [92]:
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for input_ids, attention_mask, labels in tqdm(train_loader, desc="Epoch {}".format(epoch + 1)):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predicted = torch.max(logits, 1)

        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = train_correct / train_total
    train_loss /= len(train_loader)

    print("Epoch {}: Train Loss = {:.4f}, Train Accuracy = {:.4f}".format(epoch + 1, train_loss, train_accuracy))


Epoch 1:   0%|          | 0/2109 [00:00<?, ?it/s]

RuntimeError: ignored

In [74]:
model.eval()
test_correct = 0
test_total = 0

with torch.no_grad():
    for input_ids, attention_mask, labels in tqdm(test_loader, desc="Testing"):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.last_hidden_state
        _, predicted = torch.max(logits, 1)

        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)
        test_accuracy = test_correct / test_total
print("Test Accuracy: {:.4f}".format(test_accuracy))

# Load the dataset containing music preferences and sentiments
dataset = pd.read_excel('musiclist (3).xlsx')

# Group the dataset by sentiment and aggregate the music preferences
sentiment_preferences = dataset.groupby('감정')['음악'].apply(list).reset_index()

# Vectorize the music preferences
vectorizer = CountVectorizer()
music_vectors = vectorizer.fit_transform(sentiment_preferences['음악'].apply(' '.join))

# Compute cosine similarity matrix between sentiment preferences
similarity_matrix = cosine_similarity(music_vectors)

def recommend_music(sentiment, top_n=5):
    if sentiment not in sentiment_preferences['감정'].values:
        print("Sentiment '{}' not found in the dataset.".format(sentiment))
        return []  # Return an empty list if sentiment not found

    try:
        sentiment_index = sentiment_preferences[sentiment_preferences['감정'] == sentiment].index[0]
        similarity_scores = similarity_matrix[sentiment_index]
        similar_sentiments = sorted(enumerate(similarity_scores), key=lambda x: x[1], reverse=True)
        recommended_sentiments = [sentiment_preferences['감정'][i] for i, _ in similar_sentiments[1:top_n+1]]
        recommended_music = [music for sentiments in recommended_sentiments for music in sentiment_preferences[sentiments]['음악']]
        return recommended_music[:top_n]
    except KeyError as e:
        print("Error: KeyError occurred. Make sure the sentiment column and music preferences are correctly defined in the dataset.")
        return []

# Example usage:
sentiment = '행복'
recommendations = recommend_music(sentiment, top_n=5)

# Write recommendations to a file
with open('RecommendedMusic.txt', 'w') as file:
    file.write("Recommended music for {}: \n".format(sentiment))
    for music in recommendations:
        file.write(music + "\n")

Testing:   0%|          | 0/528 [00:00<?, ?it/s]

AttributeError: ignored

In [ ]:
def recommend_music(sentiment, top_n=5):
    if sentiment not in sentiment_preferences['감정'].values:
        print("Sentiment '{}' not found in the dataset.".format(sentiment))
        return []  # Return an empty list if sentiment not found

    try:
        sentiment_index = sentiment_preferences[sentiment_preferences['감정'] == sentiment].index[0]
        similarity_scores = similarity_matrix[sentiment_index]
        similar_sentiments = sorted(enumerate(similarity_scores), key=lambda x: x[1], reverse=True)
        recommended_sentiments = [sentiment_preferences['감정'][i] for i, _ in similar_sentiments[1:top_n+1]]
        recommended_music = [music for sentiments in recommended_sentiments for music in sentiment_preferences[sentiments]['음악']]
        return recommended_music[:top_n]
    except KeyError as e:
        print("Error: KeyError occurred. Make sure the sentiment column and music preferences are correctly defined in the dataset.")
        return []

# Example usage:
sentiment = '행복'
recommendations = recommend_music(sentiment, top_n=5)

# Write recommendations to a file
with open('RecommendedMusic.txt', 'w') as file:
    file.write("Recommended music for {}: \n".format(sentiment))
    for music in recommendations:
        file.write(music + "\n")

In [ ]:
label_map = {
    "공포": 1,
    "놀람": 2,
    "분노": 3,
    "슬픔": 4,
    "중립": 5,
    "행복": 6,
    "혐오": 7,
}

losses = []
accuracies = []

for i in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = [label_map[label] for label in y_batch]

        input_ids_batch = input_ids_batch.to(device, dtype=torch.float32)
        attention_masks_batch = attention_masks_batch.to(device, dtype=torch.float32)

        y_batch = torch.tensor(y_batch).to(device)

        y_one_hot = F.one_hot(y_batch, num_classes=len(label_map)).float()
        
        y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        
        y_pred = y_pred.view(-1, len(label_map))

        loss = CrossEntropyLoss()(y_pred, torch.argmax(y_one_hot, dim=1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
            print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

    losses.append(total_loss)
    accuracies.append(correct.float() / total)
    print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/2636 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

label_map = {
    "공포": 1,
    "놀람": 2,
    "분노": 3,
    "슬픔": 4,
    "중립": 5,
    "행복": 6,
    "혐오": 7,
}

losses = []
accuracies = []

for i in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0
    batches = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = [label_map[label] for label in y_batch]

        input_ids_batch = input_ids_batch.to(device, dtype=torch.float32)
        attention_masks_batch = attention_masks_batch.to(device, dtype=torch.float32)

        y_batch = torch.tensor(y_batch).to(device)

        y_one_hot = F.one_hot(y_batch, num_classes=len(label_map)).float()

        y_pred = model(input_ids_batch, attention_mask=attention_masks_batch)[0]

        y_pred = y_pred.view(-1, len(label_map))

        loss = CrossEntropyLoss()(y_pred, torch.argmax(y_one_hot, dim=1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y_batch).sum().item()
        total += len(y_batch)

        batches += 1
        if batches % 100 == 0:
            print("Batch Loss:", total_loss, "Accuracy:", correct / total)

    losses.append(total_loss)
    accuracies.append(correct / total)
    print("Train Loss:", total_loss, "Accuracy:", correct / total)


  0%|          | 0/2636 [00:00<?, ?it/s]

RuntimeError: ignored